# non-numerical dataframes

dataframes will be challenging for folks with dyscalculia. the [wai-adapt](https://www.w3.org/TR/adapt/) has started writing guidance for concepts like this, but dataframes are likely out of scope.

this notebook sketches an approach towards subtle visual identificaition of a table value in a distribution. before making this i realized that there is no way to explore a table of numbers with dyscalculia. i imagine i'll be thinking more about the visual, audible, and tactile aspects of numeric information.

we'll want a systematic approach where we explore how the dimensions of line, tone/weight, and, lastly, color can present numerical concepts using visual gestalts.

an interesting outcome of this approach is that scrolling activates an animation of the values. it reverses back into albers assertion that notice color in motion and when it is changing. simple concepts like this have significant potential for activiting ones participation in exploring data.

In [1]:
%%
        df = pipe(numpy.linspace(0, 360, 101), pandas.Index).rename("index").to_series().mul(numpy.pi/180).apply(lambda x: pandas.Series(dict(x=numpy.cos(x), y=numpy.sin(x))))
        style_attributes = lambda extra="": compose_left(operator.methodcaller("items"), map(F"--%s{extra}: %s".__mod__), ";".join, "{};".format)

{% set df = df.reset_index() %}
{% set summary = df.describe() %}
{% set groups = [df, df.sample(20)] %}
<style>
.dataframe {
    --height: 600px;
    display: block;
    position: relative;
    width: 100%;
    border: 5px red solid;
    {%- for col in df.columns -%}
    --d{{col}}: calc(var(--{{col}}-max) - var(--{{col}}-min));
    {%- endfor -%}
    --dx: calc(var(--x-max) - var(--x-min));
    --dy: calc(var(--y-max) - var(--y-min));
    tbody, thead {
        tr {
            border: 4px white solid;
            position: relative;
            td {
                {%- for col in df.columns -%}
                &:nth-of-type({{loop.index}}) {
                    --rel: calc((var(--{{col}}) - var(--{{col}}-min)) / var(--d{{col}}));
                }
                {%- endfor -%}
                background: linear-gradient(90deg, 
                        rgba(0,0,0,0) calc(100% * var(--rel) - .25rem), 
                        rgba(128,128,128,.5) calc(100% * var(--rel)), 
                        rgba(0,0,0,0) calc(100% * var(--rel) + .25rem)
                    );
            }
            
        }
    }
}

</style>

%%
<table class=dataframe style="{{summary.loc["min"].pipe(style_attributes("-min"))}}{{summary.loc["max"].pipe(style_attributes("-max"))}}">
    <caption>a table of sin and cosine data with visual representations in each cell.</caption>
    <thead>
     <tr>
        <th>index</th>
         {%- for k in df.columns -%}
             <th>{{k}}</th>
         {%- endfor -%}
    </tr>
    {%-for i, row in summary.iterrows() -%}
    <tr style="{{style_attributes()(row)}}">
        <th scope=row>{{row.name}}</th>
        {%- for k in df.columns -%}
        <td>{{row.loc[k]}}</td>
        {%- endfor -%}
    </tr>
    {%- endfor -%}
    </thead>
     {%- for g in groups -%}
     <tbody>
         {%- for i, row in g.iterrows() -%}
            <tr style="{{style_attributes()(row)}}">
             <th>{{loop.index}}</th>
             {%- for k in df.columns -%}
                 <td>{{row.loc[k]}}</td>
             {%- endfor -%}
            </tr>
         {%- endfor -%}
     </tbody>
     {%- endfor -%}
</table>

index,index,x,y
count,101.0,101.0,101.0
mean,180.0,0.009900990099009927,5.681783374570121e-18
std,105.48061433268201,0.7140735186932855,0.7071067811865475
min,0.0,-1.0,-1.0
25%,90.0,-0.6845471059286887,-0.6845471059286887
50%,180.0,6.123233995736766e-17,0.0
75%,270.0,0.7289686274214116,0.6845471059286887
max,360.0,1.0,1.0
1,0.0,1.0,0.0
2,3.6,0.9980267284282716,0.06279051952931337
